In [1]:
# pylint: disable=missing-module-docstring
%load_ext autoreload
%autoreload 1
%aimport simulation

# pylint: disable=wrong-import-position
from simulation.parameters import Param
from simulation.model import Model

In [2]:
param = Param(log_to_console=True, data_collection_period=25)
model = Model(param=param, run_number=0)
model.run()

0.000: Initialise model:                                                                                           
                                                                                                                   
{   'arrival_dist': {   'asu': {   'neuro': <sim_tools.distributions.Exponential object at 0x723323387cb0>,        
                                   'other': <sim_tools.distributions.Exponential object at 0x7233233f0f50>,        
                                   'stroke': <sim_tools.distributions.Exponential object at 0x7233233f1090>,       
                                   'tia': <sim_tools.distributions.Exponential object at 0x7233233ecc30>},         
                        'rehab': {   'neuro': <sim_tools.distributions.Exponential object at 0x7233233ecd60>,      
                                     'other': <sim_tools.distributions.Exponential object at 0x7233d8d9fe30>,      
                                     'stroke': <sim_tools.distributions.

In [3]:
model.audit_list

[{'time': 0, 'asu_occupancy': 0, 'rehab_occupancy': 0},
 {'time': 1, 'asu_occupancy': 0, 'rehab_occupancy': 0},
 {'time': 2, 'asu_occupancy': 2, 'rehab_occupancy': 0},
 {'time': 3, 'asu_occupancy': 3, 'rehab_occupancy': 0},
 {'time': 4, 'asu_occupancy': 4, 'rehab_occupancy': 0},
 {'time': 5, 'asu_occupancy': 4, 'rehab_occupancy': 0},
 {'time': 6, 'asu_occupancy': 4, 'rehab_occupancy': 0},
 {'time': 7, 'asu_occupancy': 5, 'rehab_occupancy': 0},
 {'time': 8, 'asu_occupancy': 7, 'rehab_occupancy': 0},
 {'time': 9, 'asu_occupancy': 6, 'rehab_occupancy': 1},
 {'time': 10, 'asu_occupancy': 7, 'rehab_occupancy': 2},
 {'time': 11, 'asu_occupancy': 7, 'rehab_occupancy': 2},
 {'time': 12, 'asu_occupancy': 4, 'rehab_occupancy': 2},
 {'time': 13, 'asu_occupancy': 5, 'rehab_occupancy': 2},
 {'time': 14, 'asu_occupancy': 3, 'rehab_occupancy': 2},
 {'time': 15, 'asu_occupancy': 6, 'rehab_occupancy': 2},
 {'time': 16, 'asu_occupancy': 4, 'rehab_occupancy': 1},
 {'time': 17, 'asu_occupancy': 5, 'rehab_